In [8]:
# Import packages

import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os, sys
import time
import torch
torch.backends.cudnn.benchmark = True # False: reproducible convolution but slower

import tv
import tv_2d
import tv_pyTorch

import tv_operators
import tv_operators_pyTorch

from tests import test_transpose

%autosave 30
%load_ext autoreload
%autoreload 2

Autosaving every 30 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## General definitions

- 2D Image $f_{i,j}$ with $0 \leq (i,j) \leq N-1$

- Row difference: $r_{i,j} = f_{i+1,j}-f{i,j}$

- Column difference: $c_{i,j} = f_{i,j+1}-f{i,j}$

- Example matrix: 

$
A = \begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Upwind


- $TVU(f) = \sum_{i,j = 0}^{N-2} \sqrt{r_{i,j}^2+c_{i,j}^2} = \sum_{i,j = 0}^{N-2} G^U_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^U_{i,j} = \sqrt{r_{i,j}^2+c_{i,j}^2}$

- $[\partial TVU(f)]_{p,q} = \partial_{p,q}(G^U_{p,q}) + \partial_{p,q}(G^U_{p-1,q}) + \partial_{p,q}(G^U_{p,q-1}) = - \frac{r_{p,q}+c_{p,q}}{G^U_{p,q}} +\frac{r_{p-1,q}}{G^U_{p-1,q}} + \frac{c_{p,q-1}}{G^U_{p,q-1}}$

- $TVU(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & -1 & 2+\sqrt{2} & -1/\sqrt{2} & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$

## TV Downwind


- $TVD(f) = \sum_{i,j = 1}^{N-1} \sqrt{r_{i-1,j}^2+c_{i,j-1}^2} = \sum_{i,j = 0}^{N-2} \sqrt{r_{i,j+1}^2+c_{i+1,j}^2} =  \sum_{i,j = 0}^{N-2} G^D_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^D_{i,j} = \sqrt{r_{i,j+1}^2+c_{i+1,j}^2}$

- $[\partial TVD(f)]_{p,q} = \partial_{p,q}(G^D_{p-1,q-1}) + \partial_{p,q}(G^D_{p-1,q}) + \partial_{p,q}(G^D_{p,q-1}) = \frac{r_{p-1,q}+c_{p,q-1}}{G^D_{p-1,q-1}} - \frac{c_{p,q}}{G^D_{p-1,q}} - \frac{r_{p,q}}{G^D_{p,q-1}}$

- $TVD(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1/\sqrt{2} & 0 & 0 \\
0 & -1/\sqrt{2} & 2+\sqrt{2} & -1 & 0 \\
0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Mixture


- $TVM(f) = (TVD(f)+TVU(f))/2$

- Not differentiable everywhere, isotropic.

- Can't be written from the standard TV definition using a numerical discretization of $\left( \frac{\partial f}{\partial x}\right) ^2$.

- $TVM(A) = 2+\sqrt{2}$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 0 & 0 \\
0 & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 2+\sqrt{2} & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 0 \\
0 & 0 & -\frac{1+\sqrt{2}}{2\sqrt{2}} & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Hybrid 

- $TVH(f) = \frac{1}{\sqrt{2}}\sum_{i,j = 0}^{N-2} \sqrt{r_{i,j}^2 + r_{i,j+1}^2 + c_{i,j}^2 + +c_{i+1,j}^2} = \frac{1}{\sqrt{2}} \sum_{i,j = 0}^{N-2} G^H_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^H_{i,j} = \sqrt{r_{i,j}^2 + c_{i,j}^2 + r_{i,j+1}^2+c_{i+1,j}^2}$

- $[\partial TVH(f)]_{p,q} = \partial_{p,q}(G^H_{p,q}) + \partial_{p,q}(G^H_{p,q-1})+ \partial_{p,q}(G^H_{p-1,q})  + \partial_{p,q}(G^H_{p-1,q-1})  = -\frac{r_{p,q}+c_{p,q}}{G^H_{p,q}} + \frac{c_{p,q-1}-r_{p,q}}{G^H_{p,q-1}} + \frac{r_{p-1,q}-c_{p,q}}{G^H_{p-1,q}} + \frac{r_{p-1,q}+c_{p,q-1}}{G^H_{p-1,q-1}}$

- $TVM(A) = 4.0$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -2 & 0 & 0 \\
0 & -2 & 8 & -2 & 0 \\
0 & 0 & -2 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{pmatrix}
$


## TV Central

- $TVC(f) = \frac{1}{2} \sum_{i,j = 0}^{N-3} \sqrt{(r_{i+1,j}+r_{i,j})^2 + (c_{i,j+1} +c_{i,j})^2} = \frac{1}{2} \sum_{i,j = 0}^{N-3} G^C_{i,j}$

- Not differentiable everywhere, isotropic.

- $G^C_{i,j} = \sqrt{(r_{i+1,j}+r_{i,j})^2 + (c_{i,j+1} +c_{i,j})^2}$

- $[\partial TVC(f)]_{p,q} = \partial_{p,q}(G^C_{p,q}) + \partial_{p,q}(G^C_{p,q-1})+ \partial_{p,q}(G^C_{p-1,q})  + \partial_{p,q}(G^C_{p,q-2})  + \partial_{p,q}(G^C_{p-2,q})  = -\frac{r_{p+1,q}+r_{p,q}+c_{p,q+1}+c_{p,q}}{G^C_{p,q}} - 0 - 0+ \frac{r_{p-1,q}+r_{p-2,q}}{G^C_{p-2,q}} + \frac{c_{p,q-1}+c_{p,q-2}}{G^C_{p,q-2}}$

- $TVM(A) = 2.0$ and the following matrix is a subgradient:

$
\begin{pmatrix}
0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
-1 & 0 & 4 & 0 & -1 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 \\
\end{pmatrix}
$

# Implementation tests

In [4]:
# Check that 4 different implementations are equal for each scheme

# Options are 'downwind', 'upwind', 'centered', 'hybrid'
tv_scheme = 'hybrid' # Scheme of discretization used to compute the TV

# N = 5
# img = np.zeros([N,N])
# img[N//2, N//2] = 1.0
# plt.imshow(img)
img = np.random.rand(5000,5000)

print('Checking TV values for discretization scheme: '+str(tv_scheme))

print('\n')
tic = time.time()
(this_tv, G) = eval('tv.tv_'+tv_scheme+'(img)')
toc = time.time()
print('CPU took: '+str(toc-tic)+' s')
print('TV = '+str(this_tv))

tic = time.time()
(this_tv2, G2) = eval('tv_pyTorch.tv_'+tv_scheme+'(img)')
toc = time.time()
print('\nGPU (PyTorch) took: '+str(toc-tic)+' s')
print('TV = '+str(this_tv2))

img2 = np.reshape(img, (1,1,)+img.shape)
tic = time.time()
this_tv3 = eval('tv_operators.compute_L21_norm(tv_operators.D_'+tv_scheme+'(img2))')
toc = time.time()
print('\nCPU operators took: '+str(toc-tic)+' s')
print('TV = '+str(this_tv3))

tic = time.time()
this_tv4 = eval('tv_operators_pyTorch.compute_L21_norm(tv_operators_pyTorch.D_'+tv_scheme+'(img2, return_pytorch_tensor=True))')
toc = time.time()
print('\nGPU (PyTorch) operators took: '+str(toc-tic)+' s')
print('TV = '+str(this_tv4))

Checking TV values for discretization scheme: hybrid


CPU took: 4.484408378601074 s
TV = 13563121.480085619


RuntimeError: Expected 5-dimensional input for 5-dimensional weight [1, 1, 1, 2, 1], but got 4-dimensional input of size [1, 1, 5000, 5000] instead

In [ ]:
N = 5
Nz = 3
img = np.random.rand(N,N)
img_3d = np.tile(img, [Nz, 1, 1])
error = 1e-6

tv1, G1 = tv.tv_downwind(img_3d)
tv1b, G1b = tv_pyTorch.tv_downwind(img_3d)
tv2, G2 = tv_2d.tv_downwind(img)
print('TV downwind: '+str(abs(tv1/(Nz-2) - tv2)/tv2 < error))
print('\t GPU: '+str(abs(tv1 - tv1b)/tv1 < error)+', '+str(np.sum(abs(G1 - G1b)) / np.sum(abs(G1)) < error))

tv1, G1 = tv.tv_upwind(img_3d)
tv1b, G1b = tv_pyTorch.tv_upwind(img_3d)
tv2, G2 = tv_2d.tv_upwind(img)
print('TV upwind: '+str(abs(tv1/(Nz-1) - tv2)/tv2 < error))
print('\t GPU: '+str(abs(tv1 - tv1b)/tv1 < error)+', '+str(np.sum(abs(G1 - G1b)) / np.sum(abs(G1)) < error))

tv1, G1 = tv.tv_centered(img_3d)
tv2, G2 = tv_2d.tv_centered(img)
print('TV centered: '+str(abs(tv1/(Nz-2) - tv2)/tv2 < error))
tv1b, G1b = tv_pyTorch.tv_centered(img_3d)
print('\t GPU: '+str(abs(tv1 - tv1b)/tv1 < error)+', '+str(np.sum(abs(G1 - G1b))/ np.sum(abs(G1)) < error))

print('TV hybrid:')
tv1, G1 = tv.tv_hybrid(img_3d)
tv1b, G1b = tv_pyTorch.tv_hybrid(img_3d)
print('\t GPU: '+str(abs(tv1 - tv1b)/tv1 < error)+', '+str(np.sum(abs(G1 - G1b)) / np.sum(abs(G1)) < error))

In [ ]:
N = 100
Nz = 5
M = 3
error = 1e-6
n_test = 20
reg_time = np.power(2.0, -5)

np.random.seed(0)
img = np.random.rand(1, 1, N, N)
img_3d = np.tile(img, [Nz, 1, 1, 1])

print('TV upwind:')
D_img_3d = tv_operators.D_upwind(img_3d)
D_img = tv_operators.D_upwind(img)
print('\t D, 2D vs 3D: '+str(np.prod(D_img_3d[0,0:2,:,:,:] == D_img)>0))
tv1 = tv_operators.compute_L21_norm(D_img)
tv2 = tv_operators.compute_L21_norm(D_img_3d[:1,0:2,:,:,:])
print('\t TV value, 2D vs 3D: '+str(np.abs(tv1-tv2)/tv1 < error))

D_T_D_img_3d = tv_operators.D_T_upwind(D_img_3d)
D_T_D_img = tv_operators.D_T_upwind(D_img)
print('\t D_T, 2D vs 3D: '+str(np.prod(D_T_D_img_3d[0] == D_T_D_img)>0))

tv3, G3 = tv.tv_upwind(img.squeeze())
tv4 = tv_operators.compute_L21_norm(D_img)
print('\t TV value 2D, vs non-operator form: '+str(np.abs(tv4-tv3)/tv4 < error))

img_3d = np.random.rand(Nz, 1, N,N)
D_img_3d = tv_operators.D_upwind(img_3d)
tv5, G5 = tv.tv_upwind(img_3d.squeeze())
tv6 = tv_operators.compute_L21_norm(D_img_3d)
print('\t TV value 3D, vs non-operator form: '+str(np.abs(tv6-tv5)/tv6 < error))
D = lambda x: tv_operators.D_upwind(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_upwind(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 1)
D = lambda x: tv_operators.D_upwind(np.transpose(np.reshape(x, (1,)+x.shape), [1, 0, 2, 3]))
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 5)

print('\t TV value 2D & time: ')
D_T = lambda y: tv_operators.D_T_upwind(y, reg_time = reg_time)
D = lambda x: tv_operators.D_upwind(np.reshape(x, (1,)+x.shape), reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 1, M = M)

print('\t TV value 3D & time: ')
D_T = lambda y: tv_operators.D_T_upwind(y, reg_time = reg_time)
D = lambda x: tv_operators.D_upwind(x, reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 5, M = M)


In [ ]:
N = 100
Nz = 5
M = 3
error = 1e-6
n_test = 20
reg_time = np.power(2.0, -5)

np.random.seed(0)
img = np.random.rand(1, 1, N, N)
img_3d = np.tile(img, [Nz, 1, 1, 1])

print('TV downwind:')
D_img_3d = tv_operators.D_downwind(img_3d)
D_img = tv_operators.D_downwind(img)
print('\t D, 2D vs 3D: '+str(np.prod(D_img_3d[1,0:2,:,:,:] == D_img)>0))
# print(D_img_3d[0])
# print(D_img)
tv1 = tv_operators.compute_L21_norm(D_img)
tv2 = tv_operators.compute_L21_norm(D_img_3d[1:2,0:2,:,:,:])
print('\t TV value, 2D vs 3D: '+str(np.abs(tv1-tv2)/tv1 < error))

D_T_D_img_3d = tv_operators.D_T_downwind(D_img_3d)
D_T_D_img = tv_operators.D_T_downwind(D_img)
print('\t D_T, 2D vs 3D: '+str(np.prod(D_T_D_img_3d[1] == D_T_D_img)>0))

tv3, G3 = tv.tv_downwind(img.squeeze())
tv4 = tv_operators.compute_L21_norm(D_img)
print('\t TV value 2D, vs non-operator form: '+str(np.abs(tv4-tv3)/tv4 < error))

img_3d = np.random.rand(Nz, 1, N,N)
D_img_3d = tv_operators.D_downwind(img_3d)
tv5, G5 = tv.tv_downwind(img_3d.squeeze())
tv6 = tv_operators.compute_L21_norm(D_img_3d)
print('\t TV value 3D, vs non-operator form: '+str(np.abs(tv6-tv5)/tv6 < error))
D = lambda x: tv_operators.D_downwind(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_downwind(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 1)
D = lambda x: tv_operators.D_downwind(np.transpose(np.reshape(x, (1,)+x.shape), [1, 0, 2, 3]))
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 5)

print('\t TV value 2D & time: ')
D_T = lambda y: tv_operators.D_T_downwind(y, reg_time = reg_time)
D = lambda x: tv_operators.D_downwind(np.reshape(x, (1,)+x.shape), reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 1, M = M)

print('\t TV value 3D & time: ')
D_T = lambda y: tv_operators.D_T_downwind(y, reg_time = reg_time)
D = lambda x: tv_operators.D_downwind(x, reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 5, M = M)


In [101]:
N = 10
Nz = 4
M = 5
error = 1e-6
n_test = 20
reg_time = np.power(2.0, -5)

np.random.seed(0)
img = np.random.rand(1, 1, N, N)
img_3d = np.tile(img, [Nz, 1, 1, 1])

print('TV centered:')
D_img_3d = tv_operators.D_centered(img_3d)
D_img = tv_operators.D_centered(img)
print('\t D, 2D vs 3D: '+str(np.prod(D_img_3d[1,0:2,:,:,:] == D_img)>0))

tv1 = tv_operators.compute_L21_norm(D_img)
tv2 = tv_operators.compute_L21_norm(D_img_3d[1:2,0:2,:,:,:])
print('\t TV value, 2D vs 3D: '+str(np.abs(tv1-tv2)/tv1 < error))

D_T_D_img_3d = tv_operators.D_T_centered(D_img_3d)
D_T_D_img = tv_operators.D_T_centered(D_img)
print('\t D_T, 2D vs 3D: '+str(np.prod(D_T_D_img_3d[1] == D_T_D_img)>0))

tv3, G3 = tv.tv_centered(img.squeeze())
tv4 = tv_operators.compute_L21_norm(D_img)
print('\t TV value 2D, vs non-operator form: '+str(np.abs(tv4-tv3)/tv4 < error))

img_3d = np.random.rand(Nz, 1, N,N)
D_img_3d = tv_operators.D_centered(img_3d)
tv5, G5 = tv.tv_centered(img_3d.squeeze())
tv6 = tv_operators.compute_L21_norm(D_img_3d)
print('\t TV value 3D, vs non-operator form: '+str(np.abs(tv6-tv5)/tv6 < error))
D = lambda x: tv_operators.D_centered(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_centered(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 1)
D = lambda x: tv_operators.D_centered(np.transpose(np.reshape(x, (1,)+x.shape), [1, 0, 2, 3]))
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 5)

print('\t TV value 2D & time: ')
D_T = lambda y: tv_operators.D_T_centered(y, reg_time = reg_time)
D = lambda x: tv_operators.D_centered(np.reshape(x, (1,)+x.shape), reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 1, M = M)

print('\t TV value 3D & time: ')
D_T = lambda y: tv_operators.D_T_centered(y, reg_time = reg_time)
D = lambda x: tv_operators.D_centered(x, reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 5, M = M)

TV centered:
	 D, 2D vs 3D: True
	 TV value, 2D vs 3D: True
	 D_T, 2D vs 3D: True
	 TV value 2D, vs non-operator form: True
	 TV value 3D, vs non-operator form: True
Transposition test: PASSED
Transposition test: PASSED
	 TV value 2D & time: 
Transposition test: PASSED
	 TV value 3D & time: 
Transposition test: PASSED


In [151]:
N = 7
Nz = 3
M = 2
error = 1e-6
n_test = 20
reg_time = np.power(2.0, -5)

np.random.seed(0)
img = np.random.rand(1, 1, N, N)
img_3d = np.tile(img, [Nz, 1, 1, 1])

print('TV hybrid:')
D_img_3d = tv_operators.D_hybrid(img_3d)
D_img = tv_operators.D_hybrid(img)
print('\t D, 2D vs 3D: '+str(np.prod(D_img_3d[1,0:4,:,:,:] == D_img)>0))

tv1 = tv_operators.compute_L21_norm(D_img)
tv2 = tv_operators.compute_L21_norm(D_img_3d[1:2,0:4,:,:,:])
print('\t TV value, 2D vs 3D: '+str(np.abs(tv1-tv2)/tv1 < error))

D_T_D_img_3d = tv_operators.D_T_hybrid(D_img_3d)
D_T_D_img = tv_operators.D_T_hybrid(D_img)
print('\t D_T, 2D vs 3D: '+str(np.prod(D_T_D_img_3d[1] == D_T_D_img)>0))

tv3, G3 = tv.tv_hybrid(img.squeeze())
tv4 = tv_operators.compute_L21_norm(D_img)
print('\t TV value 2D, vs non-operator form: '+str(np.abs(tv4-tv3)/tv4 < error))

img_3d = np.random.rand(Nz, 1, N,N)
D_img_3d = tv_operators.D_hybrid(img_3d)
tv5, G5 = tv.tv_hybrid(img_3d.squeeze())
tv6 = tv_operators.compute_L21_norm(D_img_3d)
print('\t TV value 3D, vs non-operator form: '+str(np.abs(tv6-tv5)/tv6 < error))
D = lambda x: tv_operators.D_hybrid(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_hybrid(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 1)
D = lambda x: tv_operators.D_hybrid(np.transpose(np.reshape(x, (1,)+x.shape), [1, 0, 2, 3]))
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test, nz = 5)

print('\t TV value 2D & time: ')
D_T = lambda y: tv_operators.D_T_hybrid(y, reg_time = reg_time)
D = lambda x: tv_operators.D_hybrid(np.reshape(x, (1,)+x.shape), reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 1, M = M)

print('\t TV value 3D & time: ')
D_T = lambda y: tv_operators.D_T_hybrid(y, reg_time = reg_time)
D = lambda x: tv_operators.D_hybrid(x, reg_time = reg_time)
res = test_transpose(D, D_T, tolerance = tolerance, n_test = 1, nz = 5, M = M)

TV hybrid:
	 D, 2D vs 3D: False
	 TV value, 2D vs 3D: False
	 D_T, 2D vs 3D: True
	 TV value 2D, vs non-operator form: True
	 TV value 3D, vs non-operator form: True
Transposition test: PASSED
Transposition test: PASSED
	 TV value 2D & time: 
Transposition test: PASSED
	 TV value 3D & time: 
Transposition test: PASSED


In [ ]:
# Check that the primal/dual operator implementations are actually transpose of each other



tolerance = 1e-3
n_test = 50
np.random.rand(0)

D = lambda x: tv_operators.D_upwind(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_upwind(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators.D_downwind(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_downwind(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators.D_hybrid(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_hybrid(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators.D_centered(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators.D_T_centered(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators_pyTorch.D_upwind(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators_pyTorch.D_T_upwind(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators_pyTorch.D_downwind(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators_pyTorch.D_T_downwind(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators_pyTorch.D_hybrid(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators_pyTorch.D_T_hybrid(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

D = lambda x: tv_operators_pyTorch.D_centered(np.reshape(x, (1,1,)+x.shape))
D_T = lambda y: tv_operators_pyTorch.D_T_centered(y)
test_transpose(D, D_T, tolerance = tolerance, n_test = n_test)

# Operators in algorithm.py


